# Day 12

## Part 1

- In the first section:
    - operation springs: `.`
    - damaged springs: `#`
    - unknown springs: `?`
- In the section section the lengths of contiguous damaged springs are listed
- It's a nonogram! or a picross?? whatever it's called... Well at least a single row of each.
- Find out how many solutions there are to each row. The sum of all rows is the solution



In [84]:
from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    ParseConfig, parse_from_file, markdown
)

In [137]:
parser = ParseConfig('\n', ParseConfig(' ', [
    str,
    ParseConfig(',', int)
]))

input_values = parse_from_file('puzzle_input\\day_12.txt', parser)

In [162]:
def count_solutions(pattern: str, sequence: list[int], debug: bool = False):
    """finds the number of solutions possible from a pattern and sequence"""
    # find all the unknowns
    unknowns = []
    possible = None
    for index, char in enumerate(pattern):
        if char == '?':
            unknowns.append(index)
            if possible is None:
                possible = 1
            else:
                possible *= 2

    if possible is None:
        return 0

    # test all possible sequences
    key = tuple(sequence)
    total_damaged = sum(sequence)
    solutions = 0
    value_length = len(bin(possible)) - 2

    for settings in range(2 * possible):
        # generate pattern
        test_pattern = ''
        set_values = f'{settings:0{value_length}b}'
        index = 0
        for char in pattern:
            if char != '?':
                test_pattern += char
            else:
                if set_values[index] == '0':
                    test_pattern += '.'
                else:
                    test_pattern += '#'
                index += 1
        
        # check if there's no point in trying because we don't have the right
        # number of damaged
        if test_pattern.count('#') != total_damaged:
            continue

        # find the sequence
        test_sequence = []
        previous = None
        for char in test_pattern:
            if char == '#':
                if previous != char:
                    test_sequence.append(1)
                else:
                    test_sequence[-1] += 1
            previous = char

        if tuple(test_sequence) == key:
            solutions += 1

    if debug:
        print(pattern, sequence, solutions)

    return solutions

example = '?###????????', [3,2,1]

print(count_solutions(*example))

10


In [87]:
total_solutions = 0
for pattern, sequence in tqdm(input_values):
    total_solutions += count_solutions(pattern, sequence)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:13<00:00, 75.57it/s]


In [88]:
markdown(
    '### Solution',
    f'The total number of possible solutions is: {total_solutions}'
)

### Solution
The total number of possible solutions is: 6958

## Part 2

- Every pattern is actually the listed pattern 5 times delimted by a `?`
- The sequence is also the listed sequence 5 times in a row

I think we can break this down by seeing how many combinations there are for a sequence appended, prepended and prepended and appended by a `?`. Let's try!

Testing with the examples it looks like we can find the total number of solutions with the following:

$$
n_{total} = n_{single} \left( \frac{n_{double}}{n_{single}} \right)^4
$$
Where:
- $n_{single}$ is the number of solutions to the base pattern
- $n_{double}$ is the number of solutions to the base pattern repeated with a `?` between the sections and the base sequence repeated twice.

In [165]:
total_unfolded_solutions = 0
for pattern, sequence in tqdm(input_values):
    solutions = count_solutions(pattern, sequence)
    double_solutions = count_solutions('?'.join([pattern] * 2), sequence + sequence)
    solution_ratio = double_solutions // solutions
    for _ in range(4):
        solutions *= solution_ratio
    total_unfolded_solutions += solutions

  0%|          | 1/1000 [00:03<1:04:40,  3.88s/it]


KeyboardInterrupt: 

This was going to take several hours to run so I'd better think of a faster solution

In [239]:
def count_solutions_2(pattern: str, sequence: list[int]) -> int:
    """a different approach to finding the number of possiblities"""
    # determine what actually needs to match
    checks = []
    for index, char in enumerate(pattern):
        if char != '?':
            checks.append(index)

    # now generate all possible patterns from the sequence and check them
    starts = []
    index = 0
    for width in sequence:
        starts.append(index)
        index += width + 1
    
    solutions = 0
    offsets = [*starts]
    max_index = len(offsets) - 1
    max_length = len(pattern)
    index = max_index
    while index >= 0:
        while offsets[-1] + sequence[-1] <= max_length:
            if index < max_index:
                index += 1
                continue

            # catch any that have no checks
            if len(checks) == 0:
                solutions += 1
            else:
                # build pattern and test
                test_pattern = ''
                pointer = 0
                for start, width in zip(offsets, sequence):
                    test_pattern += '.'*(start - pointer)
                    test_pattern += '#'*(width)
                    pointer = start + width
                    # stop adding more if we've passed the last check index
                    if len(test_pattern) > checks[-1]:
                        break
                else:
                    test_pattern += '.'*(1 + checks[-1] - len(test_pattern))
                # print(test_pattern, pattern, offsets, index)

                # else check what we need to match
                if all([
                    test_pattern[check] == pattern[check] for check in checks
                ]):
                    solutions += 1

            offsets[index] += 1

        # if we hit the end of the pattern increase the index and work out
        # where to start next
        index -= 1
        offsets[index] += 1
        for reset_index in range(index + 1, max_index + 1):
            offsets[reset_index] = offsets[reset_index - 1] + sequence[reset_index - 1] + 1
    
    return solutions

print(example)
print(count_solutions_2(*example))


('?###????????', [3, 2, 1])
10


In [240]:
total_unfolded_solutions = 0
for pattern, sequence in tqdm(input_values):
    solutions = count_solutions_2(pattern, sequence)
    double_solutions = count_solutions_2('?'.join([pattern] * 2), sequence + sequence)
    solution_ratio = double_solutions // solutions
    for _ in range(4):
        solutions *= solution_ratio
    total_unfolded_solutions += solutions

100%|██████████| 1000/1000 [01:57<00:00,  8.49it/s]


In [241]:
markdown(
    '### Solution',
    'The total number of possible unfolded solutions is: '
    f'{total_unfolded_solutions}'
)

### Solution
The total number of possible unfolded solutions is: 3443361720254